### Идея
Большинство формул нашего мира как физического так других сфер состоит из перемножения, либо деления, либо их кобминации.  
Как пример ниже формулы:  
F=ma, E=mgh, E=mw^2/2, I=q/t, A=Fs, P=F/S IMT=m/l, p=mw, Cm=n/V, n=mM, n=N/Na.  
В более сложных процесах как биохимические реакции, расчет кинетики реакции, уравнение нормального распределения,.... всплывает либо логорифмирование либо возведение экспоненты в один из признаков или их групповую комбинацию.  
В данной работе я использую данную эвристику для генерации дополнительных признаков. В качестве базовых признаков я беру следующие комбинации:
*1) умножение по 3  
2) умножение по 2  
3) делений 2 чисел  
4) умножение двух и деление на третье
5) возведение e в степень по признаку
6) логарифм признака
7) комбинация из 2х с возведениему exp в степень по одному признаку.

В дальнейшем можно создавать комбинации из комбинаций например в начале взять логарифм признака а потом перемножить получившиеся логорифмы признаков со всеми признаками в парах или тройках или же перемножив три признака взять логарифм их произведения и тд и тп

In [14]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from itertools import *

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [42]:


# Load datasets
df = pd.read_csv("/kaggle/input/test-file/tested.csv")


In [31]:
df.head()

,Pclass,Age,SibSp,Fare
0,3,34.5,0,7.8292
1,3,47.0,1,7.0000
2,2,62.0,0,9.6875
3,3,27.0,0,8.6625
4,3,22.0,1,12.2875


In [43]:
df = df[['Pclass','Age', 'SibSp', 'Fare']]
df.dropna( inplace=True)

In [44]:
df.shape[0]

331

In [45]:
class Generator():


    def __init__(self, df):
        df.self = df

    def multiply(self, by=2, mode='with_priviouse'):
        multiply_list = []
        for name_col in combinations(df.self.columns, by):
            multiply_list.append(name_col)

        for tup in multiply_list:
            df[str(tup)] = pd.Series(np.ones([df.self.shape[0]]))
            for i in range(0,by):
                df[str(tup)] *= df[tup[i]]
                
        if mode == 'with_priviouse':
            return df


    def random_generate():
        pass

    def sub_space_generator(multiply=True, devision = True, space = 3):
        pass

   
a = Generator(df)
a.multiply()

/tmp/ipykernel_35/172498447.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.self = df
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Pclass,Age,SibSp,Fare,"('Pclass', 'Age')","('Pclass', 'SibSp')","('Pclass', 'Fare')","('Age', 'SibSp')","('Age', 'Fare')","('SibSp', 'Fare')"
0,3,34.5,0,7.8292,103.5,0.0,23.4876,0.0,270.1074,0.0000
1,3,47.0,1,7.0000,141.0,3.0,21.0000,47.0,329.0000,7.0000
2,2,62.0,0,9.6875,124.0,0.0,19.3750,0.0,600.6250,0.0000
3,3,27.0,0,8.6625,81.0,0.0,25.9875,0.0,233.8875,0.0000
4,3,22.0,1,12.2875,66.0,3.0,36.8625,22.0,270.3250,12.2875
...,...,...,...,...,...,...,...,...,...,...
409,3,3.0,1,13.7750,NaN,NaN,NaN,NaN,NaN,NaN
411,1,37.0,1,90.0000,NaN,NaN,NaN,NaN,NaN,NaN
412,3,28.0,0,7.7750,NaN,NaN,NaN,NaN,NaN,NaN
414,1,39.0,0,108.9000,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
a = Generator(df)
a.multiply()

/tmp/ipykernel_35/2938336892.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[str(tup)] = pd.Series(np.ones([df.self.shape[0]]))
/tmp/ipykernel_35/2938336892.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[str(tup)] = pd.Series(np.ones([df.self.shape[0]]))
/tmp/ipykernel_35/2938336892.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a 

KeyboardInterrupt: 

In [2]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
titanic_data = pd.read_csv(url)

In [ ]:
titanic_data = titanic_data[['PassengerId','Survived','Pclass', 'Age','SibSp', 'Parch']]

In [ ]:
titanic_data.dropna(inplace=True)


In [ ]:
def multiply_3(df):
  col = df.columns
  multiply_list = []
  for name_col in combinations(col, 3):
    multiply_list.append(name_col)

  for tup in multiply_list:
    df[str(tup)]=df[tup[0]]*df[tup[1]]*df[tup[2]]

  return df



In [ ]:
def multiply_2(df):
  col = df.columns
  multiply_list = []
  for name_col in combinations(col, 2):
    multiply_list.append(name_col)

  for tup in multiply_list:
    df[str(tup)]=df[tup[0]]*df[tup[1]]

  return df

In [ ]:
def devision_2(df):
  col = df.columns
  multiply_list = []
  for name_col in permutations(col, 2):
    multiply_list.append(name_col)

  for tup in multiply_list:
    df[str(tup)]=df[tup[0]]/df[tup[1]]

  return df

In [ ]:
def devision_3(df):
  col = df.columns
  multiply_list = []
  for name_col in permutations(col, 3):
    multiply_list.append(name_col)

  for tup in multiply_list:
    df[str(tup)]=df[tup[0]]*df[tup[1]]/df[tup[2]]

  return df

In [ ]:
if i==c[-2]:


In [ ]:
def multiply_3_in_seq_v2(df):
  col = df.columns
  list_c = [a for a in range(len(df.columns))]
  for i in range(len(col)):
    df[str(df.columns[i] + df.columns[i+1] + df.columns[i+2])] = df[df.columns[i]]*df[df.columns[i+1]]*df[df.columns[i+2]]
    if i==list_c[-3]:
      return df

In [ ]:
seq=5
for el in range(seq):
  df[str(df.columns[el])]

In [ ]:
def multiply_in_seq_v2(df, seq=3):
  col = df.columns
  list_c = [a for a in range(len(df.columns))]
  for i in range(len(col)):
    name = ""
      for el in range(seq):
        name = name.join(df.columns[el])
        df[str(df.columns[i] + df.columns[i+1] + df.columns[i+2])] = df[df.columns[i]]*df[df.columns[i+1]]*df[df.columns[i+2]]

      #if i==list_c[-seq]:
        return df

In [ ]:
seq = 2
col = df.columns
list_c = [a for a in range(len(df.columns))]
for i in range(len(col)):
  name = ""
  for el in range(seq):
    name = name.join(df.columns[el])
  print(name, '    ', df.columns[el])
  #name = ""



PassengerId     
SPassengerIduPassengerIdrPassengerIdvPassengerIdiPassengerIdvPassengerIdePassengerIdd     


In [ ]:
def multiply_3_in_seq(df):
  col = df.columns
  for i, c in enumerate(col):
    df[str(df.columns[i] + df.columns[i+1] + df.columns[i+2])] = df[df.columns[i]]*df[df.columns[i+1]]*df[df.columns[i+2]]
  return df

In [ ]:
import numpy as np
a = np.array([[1,2,3],[1,4,9], [2,3,9],[3,2,1]])

In [ ]:
sklearn.metrics.pairwise.rbf_kernel(a, gamma=1.0,)

array([[1.00000000e+00, 4.24835426e-18, 3.13913279e-17, 3.35462628e-04],
       [4.24835426e-18, 1.00000000e+00, 1.35335283e-01, 5.38018616e-32],
       [3.13913279e-17, 1.35335283e-01, 1.00000000e+00, 2.17052201e-29],
       [3.35462628e-04, 5.38018616e-32, 2.17052201e-29, 1.00000000e+00]])

In [ ]:
np.exp(-40)

4.248354255291589e-18

In [ ]:
sklearn.metrics.pairwise.linear_kernel(a)

array([[14., 36., 35., 10.],
       [36., 98., 95., 20.],
       [35., 95., 94., 21.],
       [10., 20., 21., 14.]])

In [ ]:
df = pd.